In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import obb_login
from openbb import obb
from humbldata.core.utils.env import Env
import polars as pl

# obb_login()
obb.account.login(pat=Env().OBB_PAT, remember_me=True)

In [ ]:

from humbldata.toolbox.technical.mandelbrot_channel.model import (
    calc_mandelbrot_channel,
    calc_mandelbrot_channel_historical,
)


data = (
    obb.equity.price.historical(
        ["AAPL"],
        provider="yfinance",
        start_date="2020-01-01",
        end_date="2024-01-01",
        adjustment="splits_and_dividends",
    ).to_polars()
).drop(["dividends", "stock_splits"]).with_columns(pl.lit("AAPL").alias("symbol"))

mandelbrot = calc_mandelbrot_channel_historical(  # noqa: ERA001
        data,
        window="1m",
        rv_adjustment=True,
        rv_method="std",
        rv_grouped_mean=False,
        rs_method="RS",
        live_price=False,
    ).collect()
mandelbrot

## Test `calc_up_down_pct`


In [ ]:
from humbldata.portfolio.analytics.user_table.helpers import calc_up_down_pct


df = pl.DataFrame({
    "symbol": ["AAPL", "GOOGL", "MSFT"],
    "bottom_price": [5.18, 15.07, 16.24],
    "recent_price": [10.05, 20.31, 16.42],
    "top_price": [11.23, 25.17, 30.09],
})
result = calc_up_down_pct(df)
print(result)

# Setting up UserTable

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.standard_models.portfolio.analytics.user_table import UserTableQueryParams
from humbldata.core.utils.openbb_helpers import get_latest_price


symbols = UserTableQueryParams(symbol="AAPL, MSFT, NVDA").symbol
symbols



get_latest_price(symbols)

# Tesing Async


## Get Sector Async

In [44]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category, aget_equity_sector
import polars as pl

stocks = pl.Series([
    "GOOGL",  # Communication Services
    "AMZN",   # Consumer Discretionary
    "KO",     # Consumer Staples
    "XOM",    # Energy
    "JPM",    # Financials
    "JNJ",    # Health Care
    "HON",    # Industrials
    "AAPL",   # Information Technology
    "LIN",    # Materials
    "PLD",    # Real Estate
    "NEE"     # Utilities
])

# Define objects and lists for different symbol groups

# Equities
equity_symbols = ["AAPL", "NVDA", "TSLA"]

# ETFs
etf_symbols = ["XLE", "XLF", "XLU"]

# Mixed (equities and ETFs)
mixed_symbols = ["AAPL", "XLE", "XLU", "TSLA"]

# Edge cases
edge_symbols = ["BITI", "LNGG", "ETHU", "DBA"]


# test = (await aget_etf_category(symbols=["AAPL", "XLE", "DBA"], provider="yfinance")).collect()
test2 = (await aget_equity_sector(symbols=equity_symbols, provider="yfinance")).collect()
# test2 = (await aget_equity_sector(symbols=["XLE", "AAPL"], provider="yfinance")).collect()
test2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,sector
str,str
"""AAPL""","""Technology"""
"""NVDA""","""Technology"""
"""TSLA""","""Consumer Cyclical"""


In [47]:
symbols = ["CORN", 'SLV', 'DBA', "XLE", "AAPL", "FXE", "SPY", "QQQ", "BITO", "MAXI", "SATO", "BIL", "LQD", "QAI", "MNA", "CLSE"]
symbols2 = ["AAPL", "FXE", "SPY", "MAXI", "GLD"]
symbols3 = commodity_etf_symbols = ["GLD", "IAU", "SLV", "GLDM", "PDBC", "SGOL", "FTGC", "DBC", "SIVR", "USO", "IAUM", "GSG", "BCI", "PPLT", "COMT", "GLTR", "OUNZ", "BAR", "UNG", "DBA", "AAAU", "DJP", "CMDT", "KRBN", "PALL", "COM", "NBCM", "CMDY", "DBO", "BCD", "KCCA", "CPER", "HGER", "USCI", "DBB", "DBP", "GCC", "WEAT", "BNO", "UGA", "DGP", "IGLD", "COMB", "DBE", "FGDL", "CORN", "USL", "CCRV", "DJCB", "IAUF", "PLTM", "BDRY", "PDBA", "BGLD", "UCIB", "SOYB", "GRN", "PIT", "BCIM", "RENW", "SDCI", "UNL", "SHNY", "DCMT", "TAGS", "CANE", "TMET", "KEUA", "HCOM", "HARD", "EVMT", "USG", "AMPD", "DZZ", "DGZ", "KMET", "CMCI", "USOY", "ZSC", "TILL", "BWET", "DULL", "ZSB", "LNGG", "USOI", "OILK", "SLVO", "GLDI", "BOIL", "KOLD", "AGQ", "GLL", "UGL", "ZSL", "UCO", "SCO"]



df = (await aget_etf_category(symbols=edge_symbols)).collect()
df

symbol,category
str,str
"""BITI""","""Trading--Miscellaneous"""
"""LNGG""","""Equity Energy"""
"""ETHU""",null
"""DBA""","""Commodities Focused"""


In [ ]:
from humbldata.portfolio.analytics.user_table.helpers import normalize_asset_class


normalized_df = normalize_asset_class(df).select(["symbol", "category"])
normalized_df

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_etf_category


df2 = (await aget_etf_category(symbols=["UUP", "UDN", "BITI", "ETHU", "LNGG"])).collect()
df2

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_asset_class_filter


(await aget_asset_class_filter(symbols=["XLE", "XLF", "XLK"])).collect()

## Test aget_sector_filter

In [52]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.analytics.user_table.helpers import aget_sector_filter


(await aget_sector_filter(symbols=edge_symbols)).collect()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


symbol,sector
str,str
"""BITI""","""Trading--Miscellaneous"""
"""LNGG""","""Energy"""
"""ETHU""",null
"""DBA""","""Commodities Focused"""


# Test aggregate_user_table_data

In [ ]:


from humbldata.portfolio.analytics.user_table.helpers import user_table_engine
import polars as pl

out: pl.LazyFrame = (await user_table_engine(symbols=["XLU", "XLE", "AAPL"]))
out.collect()

## Test UserTableFetcher

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.portfolio.portfolio_controller import Portfolio

portfolio = Portfolio(symbols=["AAPL", "TSLA", "NVDA"], user_role="peon")
portfolio

In [ ]:
(await portfolio.analytics.user_table()).to_dict(as_series=False)

In [ ]:
%load_ext autoreload
%autoreload 2

from humbldata.core.utils.openbb_helpers import aget_latest_price


(await aget_latest_price(symbols=["AAPL", "TSLA", "NVDA"])).collect()

# Testing Sandbox

In [ ]:
import polars as pl
from humbldata.portfolio.analytics.user_table.helpers import calc_up_down_pct

data = {
    "symbol": ["AAPL", "GOOGL", "AMZN"],
    "recent_price": [150.0, 2800.0, 3000.0],
    "bottom_price": [120.0, 2700.0, 3100.0],
    "top_price": [160.0, 2900.0, 3400.0],
}

df = pl.DataFrame(data)
calc_up_down_pct(df)